In [6]:
import json
jsonl_file = "answers.jsonl"
ground_truth_file = "mplug_owl_answer_and_openai_rating.jsonl"
answers = []
ground_truth = []
with open(jsonl_file, 'r') as f:
    for line in f:
        example = json.loads(line)
        answers.append(example)
with open(ground_truth_file, 'r') as f:
    for line in f:
        example = json.loads(line)
        ground_truth.append(example)

In [7]:
answers[:3]

[{'question_id': 0,
  'prompt': 'What is the difference between the two graphs in Figure 10?',
  'text': '© copyright by Derek Bruffa, all rights reserved.',
  'answer_id': 'Hexxd67qBhmYd8Qb2Fyjyb',
  'model_id': 'alexshengzhili/llava-7b-v0',
  'metadata': {}},
 {'question_id': 1,
  'prompt': 'What is the main difference between this experiment and the one described in Section 3.1?',
  'text': 'The main difference between this experiment and the one described in Section 3.1 is the nature of the signal being analyzed. In this image, the focus is on the pressure signal ( Depicts pressure vs. time) produced by a blood pressure machine, while in Section 3.1, the emphasis was on the current ( Depicts current vs. time) produced by a device for electrical engineering experiments. The pressure signal in this image is related to a physical phenomenon, while the current signal was used to represent the flow of electric charge in the previous experiment. Both experiments demonstrate how physical 

In [12]:
ground_truth[0]

{'image_file': '1801.01974v2-Figure10-1.png',
 'id': '1801.01974v2-Figure10-1',
 'caption': 'Fig. 10. Average AUC and AUPR versus the number of synthetic ROIs generated with DSFS, 3DMM, and SHBMM according to pose and lighting effects where still-to-video FR system employs either SVM and SRC classifiers on Chokepoint (a, b) and COX-S2V (c, d) databases.',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat is the difference between the two graphs in Figure 10?'},
  {'from': 'gpt',
   'value': 'The two graphs in Figure 10 show the performance of still-to-video FR systems on the Chokepoint and COX-S2V databases, respectively. The x-axis of both graphs shows the number of synthetic ROIs generated under both pose and lighting effects, while the y-axis shows the average AUC and AUPR.'},
  {'from': 'human',
   'value': 'What are the key observations that can be made from the graphs?'},
  {'from': 'gpt',
   'value': 'The key observations that can be made from the graphs are as fol

In [ ]:
import argparse
import json
import os

import openai
import tqdm
import time
openai.api_key = os.getenv("OPENAI_KEY")
system_message = """
You are a helpful and precise assistant for checking the quality of the answer.
You are given the graph's caption, the context of the graph, the abstract, tthe title

And then you are given the question, the reference answer, and the answer generated by the model. Please
think about how helpful the model answer is to the user and rate the model answer on a scale of 0 to 10, 
where 0 is not helpful at all and 10 is very helpful. Just return the floating number between 0 and 10.
"""

def construct_input_string(index):
    content = dict()
    cur_example = data[index]
    content['title'] = cur_example['title']
    content['abstract'] = cur_example['abstract']
    content['caption'] = cur_example['caption']
    content['Question to the model'] = cur_example['q_a_pairs'][0][0]
    content['reference_answer'] = cur_example['q_a_pairs'][0][1]
    content['Candidate model answer'] = response_model[index]
    return json.dumps(content)


def get_openai_response(content_string):
    openai_response = openai.ChatCompletion.create(
                    model='gpt-4',
                    messages=[{
                        'role': 'system',
                        'content': system_message
                    }, {
                        'role': 'user',
                        'content': content_string
                    }],
                    temperature=0.2,  # TODO: figure out which temperature is best for evaluation
                    max_tokens=500,
                )['choices'][0]['message']['content']
    return openai_response

openai_responses = []
for i in range(len(data)):
    content_string = construct_input_string(i)
    openai_response = get_openai_response(content_string)
    print(openai_response)
    openai_responses.append(openai_response)
    time.sleep(2)